In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [ ]:
history = model.fit(train_data,
                    epochs=25,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 15:46:45.568326: {'loss': 1.7697291528224945, 'accuracy': 0.53484374, 'val_loss': 1.2027355424917427, 'val_accuracy': 0.628}
Epoch 1 ends at 15:51:03.498698: {'loss': 0.7630997260689736, 'accuracy': 0.7450125, 'val_loss': 0.8082613088902394, 'val_accuracy': 0.7452}
Epoch 2 ends at 15:55:24.503308: {'loss': 0.4699798501133919, 'accuracy': 0.83905625, 'val_loss': 0.7041504145807521, 'val_accuracy': 0.7871}
Epoch 3 ends at 15:59:47.980598: {'loss': 0.26791631528884174, 'accuracy': 0.905975, 'val_loss': 0.8049568158046455, 'val_accuracy': 0.7718}
Epoch 4 ends at 16:04:11.028283: {'loss': 0.177216804433614, 'accuracy': 0.93741876, 'val_loss': 0.9149952885831237, 'val_accuracy': 0.7809}
Epoch 5 ends at 16:08:34.920955: {'loss': 0.13758335811169817, 'accuracy': 0.9516, 'val_loss': 0.9725807786557326, 'val_accuracy': 0.772}
Epoch 6 ends at 16:12:57.187201: {'loss': 0.11233932321472094, 'accuracy': 0.9615375, 'val_loss': 0.8714233578010729, 'val_accuracy': 0.7921}
Epoch 7 ends a

In [13]:
model.evaluate(test_data)

157/157 [==============================].8555 - accuracy: 0.81 - 0s 52ms/step - loss: 0.8825 - accuracy: 0.82 - 0s 47ms/step - loss: 1.0508 - accuracy: 0.83 - 0s 44ms/step - loss: 1.0363 - accuracy: 0.82 - 0s 44ms/step - loss: 0.9109 - accuracy: 0.83 - 0s 43ms/step - loss: 0.9308 - accuracy: 0.83 - 0s 42ms/step - loss: 0.9559 - accuracy: 0.83 - 0s 42ms/step - loss: 0.9920 - accuracy: 0.83 - 0s 41ms/step - loss: 0.9350 - accuracy: 0.84 - 0s 41ms/step - loss: 0.9540 - accuracy: 0.84 - 0s 40ms/step - loss: 0.9013 - accuracy: 0.85 - 0s 40ms/step - loss: 0.9295 - accuracy: 0.84 - 1s 40ms/step - loss: 0.9595 - accuracy: 0.83 - 1s 39ms/step - loss: 0.9461 - accuracy: 0.83 - 1s 39ms/step - loss: 0.9248 - accuracy: 0.84 - 1s 39ms/step - loss: 0.9639 - accuracy: 0.83 - 1s 39ms/step - loss: 0.9569 - accuracy: 0.83 - 1s 38ms/step - loss: 0.9801 - accuracy: 0.84 - 1s 38ms/step - loss: 0.9871 - accuracy: 0.83 - 1s 38ms/step - loss: 0.9952 - accuracy: 0.83 - 1s 38ms/step - loss: 0.9617 - accuracy: 0.

[1.0988232183987927, 0.8213]

In [14]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))